# 03_eigenvalues_svd.ipynb

## From First Principles: Eigenvalues, Eigenvectors, and Singular Value Decomposition

This notebook explores two fundamental matrix decompositions that are crucial in machine learning: eigenvalue decomposition and singular value decomposition (SVD). We'll implement both from scratch and explore their applications.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd
%matplotlib inline

## 1. Eigenvalues and Eigenvectors

An eigenvector of a square matrix A is a non-zero vector v that, when multiplied by A, yields a scalar multiple of itself:

$Av = \lambda v$

where $\lambda$ is the corresponding eigenvalue.

In [ ]:
# Simple example: find eigenvalues and eigenvectors
A = np.array([[4, 2],
              [1, 3]])

# Using NumPy
eigenvals, eigenvecs = np.linalg.eig(A)

print(f"Matrix A:\n{A}")
print(f"Eigenvalues: {eigenvals}")
print(f"Eigenvectors (columns):\n{eigenvecs}")

# Verify: Av = λv
for i in range(len(eigenvals)):
    lambda_i = eigenvals[i]
    v_i = eigenvecs[:, i]
    
    Av = A @ v_i
    lambda_v = lambda_i * v_i
    
    print(f"\nFor eigenvalue {lambda_i:.3f} and eigenvector {v_i}:\n")
    print(f"A*v = {Av}")
    print(f"λ*v = {lambda_v}")
    print(f"Equal? {np.allclose(Av, lambda_v)}")

### Power Iteration Method

A simple algorithm to find the dominant eigenvalue and its corresponding eigenvector.

In [ ]:
def power_iteration(A, num_iterations=100, tolerance=1e-6):
    """
    Find the dominant eigenvalue and eigenvector using power iteration.
    
    Args:
        A: Square matrix
        num_iterations: Maximum number of iterations
        tolerance: Convergence tolerance
        
    Returns:
        eigenvalue: Dominant eigenvalue
        eigenvector: Corresponding eigenvector
    """
    n = A.shape[0]
    # Random initial vector
    b_k = np.random.rand(n)
    
    for _ in range(num_iterations):
        # Calculate Ab
        b_k1 = np.dot(A, b_k)
        
        # Normalize
        b_k1_norm = np.linalg.norm(b_k1)
        b_k_new = b_k1 / b_k1_norm
        
        # Check for convergence
        if np.allclose(b_k, b_k_new, atol=tolerance) or np.allclose(b_k, -b_k_new, atol=tolerance):
            break
            
        b_k = b_k_new
    
    # Calculate eigenvalue using Rayleigh quotient
    eigenvalue = (b_k.T @ A @ b_k) / (b_k.T @ b_k)
    
    return eigenvalue, b_k

# Test power iteration
dominant_eigenval, dominant_eigenvec = power_iteration(A)

print(f"Power iteration result:")
print(f"Dominant eigenvalue: {dominant_eigenval:.6f}")
print(f"Corresponding eigenvector: {dominant_eigenvec}")

# Compare with numpy result
print(f"\nNumPy result:")
idx = np.argmax(np.abs(eigenvals))
print(f"Dominant eigenvalue: {eigenvals[idx]:.6f}")
print(f"Corresponding eigenvector: {eigenvecs[:, idx]}")

### Eigenvalue Decomposition

For a diagonalizable matrix A, we can write: $A = V\Lambda V^{-1}$

Where V is the matrix of eigenvectors and Λ is the diagonal matrix of eigenvalues.

In [ ]:
# Eigenvalue decomposition
V = eigenvecs
Lambda = np.diag(eigenvals)

# Reconstruct original matrix
A_reconstructed = V @ Lambda @ np.linalg.inv(V)

print(f"Original matrix A:\n{A}")
print(f"Reconstructed A = V @ Λ @ V^(-1):\n{A_reconstructed}")
print(f"Reconstruction accurate? {np.allclose(A, A_reconstructed)}")

## 2. Singular Value Decomposition (SVD)

SVD decomposes any matrix A into: $A = UΣV^T$

Where:
- U and V are orthogonal matrices
- Σ is a diagonal matrix of singular values

SVD works for any matrix (not just square), making it more general than eigenvalue decomposition.

In [ ]:
# SVD example
A = np.array([[3, 2, 2],
              [2, 3, -2]])

# Compute SVD
U, sigma, Vt = np.linalg.svd(A)

# Note: sigma is returned as a 1D array of singular values
Sigma = np.zeros((A.shape[0], A.shape[1]))
np.fill_diagonal(Sigma, sigma)

print(f"Original matrix A (shape: {A.shape}):\n{A}")
print(f"U (shape: {U.shape}):\n{U}")
print(f"Singular values: {sigma}")
print(f"Σ (shape: {Sigma.shape}):\n{Sigma}")
print(f"V^T (shape: {Vt.shape}):\n{Vt}")

# Reconstruct original matrix
A_reconstructed = U @ Sigma @ Vt

print(f"\nReconstructed A = U @ Σ @ V^T:\n{A_reconstructed}")
print(f"Reconstruction accurate? {np.allclose(A, A_reconstructed)}")

### Relationship Between SVD and Eigenvalue Decomposition

For a matrix A, the left singular vectors (U) are the eigenvectors of $AA^T$, and the right singular vectors (V) are the eigenvectors of $A^TA$. The singular values are the square roots of the eigenvalues of $AA^T$ or $A^TA$.

In [ ]:
# Verify the relationship
A = np.array([[4, 0],
              [3, -5]])

# Compute SVD
U, sigma, Vt = np.linalg.svd(A)

# Compute eigenvalues/eigenvectors of AAT and ATA
AAT = A @ A.T
ATA = A.T @ A

eigvals_AAT, eigvecs_AAT = np.linalg.eig(AAT)
eigvals_ATA, eigvecs_ATA = np.linalg.eig(ATA)

# Sort eigenvalues and corresponding eigenvectors
idx_AAT = np.argsort(eigvals_AAT)[::-1]
idx_ATA = np.argsort(eigvals_ATA)[::-1]

eigvals_AAT = eigvals_AAT[idx_AAT]
eigvecs_AAT = eigvecs_AAT[:, idx_AAT]
eigvals_ATA = eigvals_ATA[idx_ATA]
eigvecs_ATA = eigvecs_ATA[:, idx_ATA]

print(f"Matrix A:\n{A}")
print(f"\nSVD - Left singular vectors (U):\n{U}")
print(f"SVD - Right singular vectors (V^T):\n{Vt}")
print(f"SVD - Singular values: {sigma}")

print(f"\nEigenvectors of A*A^T:\n{eigvecs_AAT}")
print(f"Eigenvectors of A^T*A:\n{eigvecs_ATA}")

print(f"\nSqrt of eigenvalues of A*A^T: {np.sqrt(np.abs(eigvals_AAT))}")
print(f"Sqrt of eigenvalues of A^T*A: {np.sqrt(np.abs(eigvals_ATA))}")
print(f"Singular values from SVD: {sigma}")

## 3. Applications in Machine Learning

### Principal Component Analysis (PCA)

PCA uses SVD to reduce dimensionality while preserving most of the variance in the data.

In [ ]:
def pca_manual(X, n_components=2):
    """
    Perform PCA using SVD.
    
    Args:
        X: Data matrix (samples x features)
        n_components: Number of principal components to keep
        
    Returns:
        X_transformed: Transformed data
        components: Principal components (loadings)
        explained_variance_ratio: Explained variance ratio for each component
    """
    # Center the data
    X_centered = X - np.mean(X, axis=0)
    
    # Compute SVD
    U, sigma, Vt = np.linalg.svd(X_centered, full_matrices=False)
    
    # Principal components are the rows of Vt
    components = Vt[:n_components]
    
    # Transform the data
    X_transformed = U[:, :n_components] * sigma[:n_components]
    
    # Calculate explained variance
    explained_variance = (sigma ** 2) / (X.shape[0] - 1)
    total_variance = np.sum(explained_variance)
    explained_variance_ratio = explained_variance / total_variance
    
    return X_transformed, components, explained_variance_ratio[:n_components]

# Create sample data
np.random.seed(42)
mean = [1, 2]
cov = [[1, 0.8], [0.8, 1]]
X = np.random.multivariate_normal(mean, cov, 100)

# Apply PCA
X_pca, components, var_ratio = pca_manual(X, n_components=2)

print(f"Original data shape: {X.shape}")
print(f"Transformed data shape: {X_pca.shape}")
print(f"Principal components:\n{components}")
print(f"Explained variance ratio: {var_ratio}")
print(f"Total variance explained: {np.sum(var_ratio):.4f}")

# Visualize
plt.figure(figsize=(12, 5))

# Original data
plt.subplot(1, 2, 1)
plt.scatter(X[:, 0], X[:, 1], alpha=0.7)
plt.title('Original Data')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

# PCA transformed data
plt.subplot(1, 2, 2)
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.7)
plt.title('PCA Transformed Data')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Matrix Rank and Low-Rank Approximation

SVD can be used to find the rank of a matrix and create low-rank approximations.

In [ ]:
def low_rank_approximation(A, rank):
    """
    Create a low-rank approximation of matrix A using SVD.
    
    Args:
        A: Input matrix
        rank: Desired rank of approximation
        
    Returns:
        A_approx: Low-rank approximation
    """
    U, sigma, Vt = np.linalg.svd(A, full_matrices=False)
    
    # Keep only the top 'rank' singular values
    sigma_truncated = np.zeros_like(sigma)
    sigma_truncated[:rank] = sigma[:rank]
    
    # Reconstruct using truncated SVD
    A_approx = U @ np.diag(sigma_truncated) @ Vt
    
    return A_approx

# Create a sample matrix
A = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

print(f"Original matrix A:\n{A}")
print(f"Rank of A: {np.linalg.matrix_rank(A)}")

# Create low-rank approximations
for r in [1, 2, 3]:
    A_approx = low_rank_approximation(A, rank=r)
    error = np.linalg.norm(A - A_approx, 'fro')  # Frobenius norm
    print(f"\nRank-{r} approximation:")
    print(f"Approximation:\n{A_approx}")
    print(f"Reconstruction error (Frobenius norm): {error:.6f}")

## 4. Applications Beyond PCA

SVD has many other applications in machine learning:

In [ ]:
# Example: Image compression using SVD

# Create a simple "image" (matrix)
def create_image():
    img = np.zeros((20, 20))
    # Add some patterns
    img[5:15, 5:15] = 1  # Square
    img[2:8, 2:8] = 0.5  # Smaller square
    return img

original_img = create_image()

# Apply SVD for compression
U, sigma, Vt = np.linalg.svd(original_img)

# Show how much of the energy is captured by top components
cumulative_energy = np.cumsum(sigma**2) / np.sum(sigma**2)

plt.figure(figsize=(15, 5))

# Original image
plt.subplot(1, 4, 1)
plt.imshow(original_img, cmap='gray', interpolation='none')
plt.title('Original Image')
plt.axis('off')

# Singular values
plt.subplot(1, 4, 2)
plt.plot(sigma, 'o-')
plt.title('Singular Values')
plt.xlabel('Index')
plt.ylabel('Value')
plt.yscale('log')

# Cumulative energy
plt.subplot(1, 4, 3)
plt.plot(cumulative_energy, 'o-')
plt.title('Cumulative Energy')
plt.xlabel('Number of Components')
plt.ylabel('Energy Captured')
plt.grid(True, alpha=0.3)

# Reconstructed image with few components
k = 5
img_compressed = U[:, :k] @ np.diag(sigma[:k]) @ Vt[:k, :]
plt.subplot(1, 4, 4)
plt.imshow(img_compressed, cmap='gray', interpolation='none')
plt.title(f'Compressed (k={k})')
plt.axis('off')

plt.tight_layout()
plt.show()

compression_error = np.linalg.norm(original_img - img_compressed, 'fro') / np.linalg.norm(original_img, 'fro')
print(f"Compression with {k} components captures {(cumulative_energy[k-1]*100):.2f}% of energy")
print(f"Relative reconstruction error: {compression_error:.4f}")

## Summary

In this notebook, we've explored:
1. Eigenvalues and eigenvectors: definitions, computation, and properties
2. Power iteration method for finding dominant eigenpairs
3. Singular Value Decomposition (SVD): definition and computation
4. Relationship between SVD and eigenvalue decomposition
5. Applications in machine learning: PCA, matrix rank, low-rank approximation, image compression

These decompositions are fundamental to understanding many machine learning algorithms, particularly those related to dimensionality reduction, matrix factorization, and understanding the structure of data.